In [ ]:
import numpy as np
from keras.models import model_from_json

# run get-networks.sh to get the keras model

In [ ]:
# Load the keras model and load the weights
with open("data/lp-detector/wpod-net_update1.json", 'r') as json_file:
            model_json = json_file.read()
wpod = model_from_json(model_json,custom_objects={})
wpod.load_weights('data/lp-detector/wpod-net_update1.h5')
weights=wpod.get_weights()

In [ ]:
# View the keras model architecture. Note, we don't use the batch normalization layers
# This architecture is slightly different from what the paper details. I implemented this one, instead of paper
wpod.summary()

In [ ]:
# Construct the same model in pytorch
# Defined model using sequential, so any parallel components are done separately
from torch import nn
import torch

# Class to represent the residual blocks since they are somewhat parallel
class ResBlock(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, inputs):
        return self.module(inputs) + inputs

#Class to represent detection block since it is parallel
class Detection(nn.Module):
    def __init__(self):
        super().__init__()
        
        # define network layers
        self.conv2 = nn.Conv2d(128,2,3)
        self.conv6 = nn.Conv2d(128,6,3)
        #self.fc = nn.Linear(16, 12) #y=x? I don't think they use this, and there are no weights corresponding to it
        self.sm = nn.Softmax()

    def forward(self, inputs):
        x1 = self.sm(self.conv2(inputs))
        #x2 = self.fc(self.conv6(inputs))
        x2 = self.conv6(inputs)
        return torch.cat(x1,x2)   #I am unsure if this is what they mean by concatenate
    
# Define model architecture
model = model = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.ReLU(),
    nn.Conv2d(16, 16, 3),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),
    nn.Conv2d(16, 32, 3),
    nn.ReLU(),    
    # ------------------------------|
    ResBlock(                      #|
        nn.Sequential(             #| 
            nn.Conv2d(16, 32, 3),  #| 
            nn.ReLU(),             #|
            nn.Conv2d(32, 32, 3),  #| 
        )                          #| 
    ),                             #|
    nn.ReLU(),                     #| 
    # ------------------------------|
    nn.MaxPool2d(2, stride=2),
    nn.Conv2d(32, 64, 3),
    nn.ReLU(),
    # ------------------------------|
    ResBlock(                      #|
        nn.Sequential(             #| 
            nn.Conv2d(64, 64, 3),  #| 
            nn.ReLU(),             #|
            nn.Conv2d(64, 64, 3),  #| 
        )                          #| 
    ),                             #|
    nn.ReLU(),                     #| 
    # ------------------------------| 
    
    # ------------------------------|
    ResBlock(                      #|
        nn.Sequential(             #| 
            nn.Conv2d(64, 64, 3),  #| 
            nn.ReLU(),             #|
            nn.Conv2d(64, 64, 3),  #| 
        )                          #| 
    ),                             #|
    nn.ReLU(),                     #| 
    # ------------------------------| 
    nn.MaxPool2d(2, stride=2),
    nn.Conv2d(64, 64, 3),             #In code not paper 
    nn.ReLU(),                        #In code not paper
    # ------------------------------|
    ResBlock(                      #|
        nn.Sequential(             #| 
            nn.Conv2d(64, 64, 3),  #| 
            nn.ReLU(),             #|
            nn.Conv2d(64, 64, 3),  #| 
        )                          #| 
    ),                             #|
    nn.ReLU(),                     #| 
    # ------------------------------|
    
    # ------------------------------|
    ResBlock(                      #|
        nn.Sequential(             #| 
            nn.Conv2d(64, 64, 3),  #| 
            nn.ReLU(),             #|
            nn.Conv2d(64, 64, 3),  #| 
        )                          #| 
    ),                             #|
    nn.ReLU(),                     #| 
    # ------------------------------| 
    nn.MaxPool2d(2, stride=2),
    nn.Conv2d(64, 128, 3),
    nn.ReLU(),                        #In code not paper
    # ------------------------------|
    ResBlock(                      #|
        nn.Sequential(             #| 
            nn.Conv2d(128, 128, 3),#| 
            nn.ReLU(),             #|
            nn.Conv2d(128, 128, 3),#| 
        )                          #| 
    ),                             #|
    nn.ReLU(),                     #| 
    # ------------------------------|
    
    # ------------------------------|
    ResBlock(                      #|
        nn.Sequential(             #| 
            nn.Conv2d(128, 128, 3),#| 
            nn.ReLU(),             #|
            nn.Conv2d(128, 128, 3),#| 
        )                          #| 
    ),                             #|
    nn.ReLU(),                     #| 
    # ------------------------------| 
    
    # ------------------------------| In code not paper |
    ResBlock(                      #|                   |
        nn.Sequential(             #|                   | 
            nn.Conv2d(128, 128, 3),#|                   |
            nn.ReLU(),             #|                   |
            nn.Conv2d(128, 128, 3),#|                   |
        )                          #|                   |
    ),                             #|                   |
    nn.ReLU(),                     #|                   |
    # ------------------------------| In code not paper |
    
    # ------------------------------| In code not paper |
    ResBlock(                      #|                   |
        nn.Sequential(             #|                   | 
            nn.Conv2d(128, 128, 3),#|                   |
            nn.ReLU(),             #|                   |
            nn.Conv2d(128, 128, 3),#|                   |
        )                          #|                   |
    ),                             #|                   |
    nn.ReLU(),                     #|                   |
    # ------------------------------| In code not paper | 
    Detection()
)

# print model architecture
print(model)

In [ ]:
#Load the weights from keras into the pytorch model
model[0].weight.data=torch.from_numpy(np.transpose(weights[0]))
model[0].bias.data=torch.from_numpy(np.transpose(weights[1]))
model[2].weight.data=torch.from_numpy(np.transpose(weights[6]))
model[2].bias.data=torch.from_numpy(np.transpose(weights[7]))
model[5].weight.data=torch.from_numpy(np.transpose(weights[12]))
model[5].bias.data=torch.from_numpy(np.transpose(weights[13]))
model[7].module[0].weight.data=torch.from_numpy(np.transpose(weights[18]))
model[7].module[0].bias.data=torch.from_numpy(np.transpose(weights[19]))
model[7].module[2].weight.data=torch.from_numpy(np.transpose(weights[24]))
model[7].module[2].bias.data=torch.from_numpy(np.transpose(weights[25]))
model[10].weight.data=torch.from_numpy(np.transpose(weights[30]))
model[10].bias.data=torch.from_numpy(np.transpose(weights[31]))
model[12].module[0].weight.data=torch.from_numpy(np.transpose(weights[36]))
model[12].module[0].bias.data=torch.from_numpy(np.transpose(weights[37]))
model[12].module[2].weight.data=torch.from_numpy(np.transpose(weights[42]))
model[12].module[2].bias.data=torch.from_numpy(np.transpose(weights[43]))
model[14].module[0].weight.data=torch.from_numpy(np.transpose(weights[48]))
model[14].module[0].bias.data=torch.from_numpy(np.transpose(weights[49]))
model[14].module[2].weight.data=torch.from_numpy(np.transpose(weights[54]))
model[14].module[2].bias.data=torch.from_numpy(np.transpose(weights[55]))
model[17].weight.data=torch.from_numpy(np.transpose(weights[60]))
model[17].bias.data=torch.from_numpy(np.transpose(weights[61]))
model[19].module[0].weight.data=torch.from_numpy(np.transpose(weights[66]))
model[19].module[0].bias.data=torch.from_numpy(np.transpose(weights[67]))
model[19].module[2].weight.data=torch.from_numpy(np.transpose(weights[72]))
model[19].module[2].bias.data=torch.from_numpy(np.transpose(weights[73]))
model[21].module[0].weight.data=torch.from_numpy(np.transpose(weights[78]))
model[21].module[0].bias.data=torch.from_numpy(np.transpose(weights[79]))
model[21].module[2].weight.data=torch.from_numpy(np.transpose(weights[84]))
model[21].module[2].bias.data=torch.from_numpy(np.transpose(weights[85]))
model[24].weight.data=torch.from_numpy(np.transpose(weights[90]))
model[24].bias.data=torch.from_numpy(np.transpose(weights[91]))
model[26].module[0].weight.data=torch.from_numpy(np.transpose(weights[96]))
model[26].module[0].bias.data=torch.from_numpy(np.transpose(weights[97]))
model[26].module[2].weight.data=torch.from_numpy(np.transpose(weights[102]))
model[26].module[2].bias.data=torch.from_numpy(np.transpose(weights[103]))
model[28].module[0].weight.data=torch.from_numpy(np.transpose(weights[108]))
model[28].module[0].bias.data=torch.from_numpy(np.transpose(weights[109]))
model[28].module[2].weight.data=torch.from_numpy(np.transpose(weights[114]))
model[28].module[2].bias.data=torch.from_numpy(np.transpose(weights[115]))
model[30].module[0].weight.data=torch.from_numpy(np.transpose(weights[120]))
model[30].module[0].bias.data=torch.from_numpy(np.transpose(weights[121]))
model[30].module[2].weight.data=torch.from_numpy(np.transpose(weights[126]))
model[30].module[2].bias.data=torch.from_numpy(np.transpose(weights[127]))
model[32].module[0].weight.data=torch.from_numpy(np.transpose(weights[132]))
model[32].module[0].bias.data=torch.from_numpy(np.transpose(weights[133]))
model[32].module[2].weight.data=torch.from_numpy(np.transpose(weights[138]))
model[32].module[2].bias.data=torch.from_numpy(np.transpose(weights[139]))
model[34].conv2.weight.data=torch.from_numpy(np.transpose(weights[144]))
model[34].conv2.bias.data=torch.from_numpy(np.transpose(weights[145]))
model[34].conv6.weight.data=torch.from_numpy(np.transpose(weights[146]))
model[34].conv6.bias.data=torch.from_numpy(np.transpose(weights[147]))